In [51]:
from ngsolve import *
from ngsolve.webgui import Draw
import random
import numpy as np

#Defining parameters
epsilon = 0.02
gamma = 50
M = 10 
maxh = 0.01 
order = 1 
tau = 0.01            
time = 0.0
T = 1


#Defining mesh and Finite element space
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
V = H1(mesh, order=1) 

# Time-dependent functions
phi = GridFunction(V)      # Current solution phi^n+1
phi_old = GridFunction(V)   # Solution phi^n
mu = GridFunction(V)        # Chemical potential mu^n+1

#Initialising phi_0 random values between -1 and 1
random_values = np.random.uniform(-1, 1, V.ndof)
phi_old.vec.FV().NumPy()[:] = random_values

#Defining test and trial functions
v = V.TestFunction()
w = V.TestFunction()
phi_trial = V.TrialFunction()
mu_trial = V.TrialFunction()



## Fully explicit scheme

Discretize $\partial_t \phi = \frac{\phi^{n+1}-\phi^n}{\tau}$, $\mu = \mu^{n+1}$, $\phi = \phi^n$ and $\nabla \phi = \nabla \phi^{n+1}$.

$$
\begin{aligned}
(\frac{\phi^{n+1}-\phi^n}{\tau},v) + (M\nabla \mu^{n+1}, \nabla v) &= 0\ \ \ \ \forall v \in \mathbb{V} \\
(\mu^{n+1},w)-(\epsilon \nabla \phi^{n+1}, \nabla w) 
&= (\gamma ((\phi^n)^3 -\phi^n),w) \ \ \ \  \forall w \in \mathbb{V} 
\end{aligned}
$$


In [52]:
#Defining parameters
epsilon = 0.02
gamma = 50
M = 10 
maxh = 0.01 
order = 1 
tau = 0.01            
T = 1

#Defining mesh and Finite element space
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
V = H1(mesh, order=1) 

# Time-dependent functions
phi = GridFunction(V)      # Current solution phi^n+1
phi_old = GridFunction(V)   # Solution phi^n
mu = GridFunction(V)        # Chemical potential mu^n+1

#Initialising phi_0 random values between -1 and 1
random_values = np.random.uniform(-1, 1, V.ndof)
phi_old.vec.FV().NumPy()[:] = random_values

#Defining test and trial functions
v = V.TestFunction()
w = V.TestFunction()
phi_trial = V.TrialFunction()
mu_trial = V.TrialFunction()



In [81]:
def FullyExplicitScheme(V,v,w,mu,phi,phi_old,phi_trial,mu_trial):

    #Storing solutions for each timestep
    phi_solutions = []
    mu_solutions = []

    #Bilinear and linear form for the first equation
    A_phi = BilinearForm(V)
    A_phi += (phi_trial * v) * dx + tau * M * grad(mu_trial) * grad(v) * dx  
    A_phi.Assemble()

    f_phi = LinearForm(V)

    #Bilinear and linear form for the second equation
    A_mu = BilinearForm(V)
    A_mu += (mu_trial * w) * dx - epsilon * grad(phi_trial) * grad(w) * dx
    A_mu.Assemble()

    f_mu = LinearForm(V)

    t = 0
    while t < T:
        t += tau

        rhs_phi =2*t-4*M
        rhs_mu= x**2+y**2-gamma*((phi_exact(t))**3-phi_exact(t))+4*epsilon


        #Right hand side of first equation
        f_phi = LinearForm(V)
        f_phi += phi_old * v * dx  +rhs_phi*v*dx # er bare 0 her originalt men definerer uansett fordi vi splitter opp diskretiserignen for phi, skriv ned
        f_phi.Assemble()

        #Solve for phi^{n+1}
        phi = GridFunction(V,name="uDG")
        phi.vec.data = A_phi.mat.Inverse(V.FreeDofs()) * f_phi.vec
        

        #Right hand side of second equation
        phi_nonlinear = CoefficientFunction(gamma * (phi_old**3 - phi_old))
        f_mu = LinearForm(V)  
        f_mu += phi_nonlinear * w * dx + rhs_mu*w*dx
        f_mu.Assemble()

        #Solve for mu^{n+1}
        mu = GridFunction(V,name="uDG")
        mu.vec.data += A_mu.mat.Inverse(V.FreeDofs()) * f_mu.vec

        
        #phi_solutions.append(phi.vec.FV().NumPy().copy())
        #mu_solutions.append(mu.vec.FV().NumPy().copy())

        phi_solutions.append(phi)
        mu_solutions.append(mu)

        #phi_solutions.append(phi)
        #mu_solutions.append(mu) 
        phi_old.vec.data = phi.vec
        #print(mu.vec.FV().NumPy().copy())

    return phi_solutions, mu_solutions


In [82]:
phi_solutions, mu_solutions = FullyExplicitScheme(V,v,w,mu,phi,phi_old,phi_trial,mu_trial)

In [88]:
phi = phi_solutions[-1]
mu = mu_solutions[-1]


 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33644e+13
 -2.33

In [56]:
def phi_exact(t):
    return t**2 +x**2 + y**2

mu_exact = x**2+y**2


In [80]:
e_h = mu_exact-mu

norm = Integrate(e_h*e_h,mesh)
norm

8.281988948432346e+24

Kilde:https://docu.ngsolve.org/nightly/i-tutorials/unit-3.1-parabolic/parabolic.html

https://docu.ngsolve.org/latest/whetting_the_appetite/navierstokes.html